In [6]:
#!pip3 install twilio

# Importar librerias

In [1]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI
import time 

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

import pandas as pd 
import requests 
from bs4 import BeautifulSoup 
from tqdm import tqdm 

from datetime import datetime 



# Armado de la URL

In [2]:
query = 'Málaga'
api_key = API_KEY_WAPI

url_clima = f'http://api.weatherapi.com/v1/forecast.json?key={api_key}&q={query}&days=1&aqi=no&alerts=no'

response = requests.get(url_clima).json() 

In [4]:
response

{'location': {'name': 'Málaga',
  'region': 'Andalucia',
  'country': 'Espagne',
  'lat': 36.7167,
  'lon': -4.4167,
  'tz_id': 'Europe/Madrid',
  'localtime_epoch': 1749643249,
  'localtime': '2025-06-11 14:00'},
 'current': {'last_updated_epoch': 1749643200,
  'last_updated': '2025-06-11 14:00',
  'temp_c': 27.1,
  'temp_f': 80.8,
  'is_day': 1,
  'condition': {'text': 'Sunny',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
   'code': 1000},
  'wind_mph': 8.1,
  'wind_kph': 13.0,
  'wind_degree': 166,
  'wind_dir': 'SSE',
  'pressure_mb': 1015.0,
  'pressure_in': 29.97,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 54,
  'cloud': 0,
  'feelslike_c': 27.5,
  'feelslike_f': 81.5,
  'windchill_c': 27.5,
  'windchill_f': 81.4,
  'heatindex_c': 27.9,
  'heatindex_f': 82.2,
  'dewpoint_c': 15.2,
  'dewpoint_f': 59.3,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 9.3,
  'gust_mph': 9.3,
  'gust_kph': 14.9},
 'forecast': {'forecastday': [{'date': '2025-06-11',
    'date

In [3]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [4]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [5]:
response['forecast']['forecastday'][0]['hour']

[{'time_epoch': 1749765600,
  'time': '2025-06-13 00:00',
  'temp_c': 19.9,
  'temp_f': 67.8,
  'is_day': 0,
  'condition': {'text': 'Clear ',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
   'code': 1000},
  'wind_mph': 3.4,
  'wind_kph': 5.4,
  'wind_degree': 88,
  'wind_dir': 'E',
  'pressure_mb': 1015.0,
  'pressure_in': 29.98,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'snow_cm': 0.0,
  'humidity': 69,
  'cloud': 0,
  'feelslike_c': 19.9,
  'feelslike_f': 67.8,
  'windchill_c': 19.9,
  'windchill_f': 67.8,
  'heatindex_c': 19.9,
  'heatindex_f': 67.8,
  'dewpoint_c': 13.9,
  'dewpoint_f': 57.1,
  'will_it_rain': 0,
  'chance_of_rain': 0,
  'will_it_snow': 0,
  'chance_of_snow': 0,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'gust_mph': 6.4,
  'gust_kph': 10.4,
  'uv': 0},
 {'time_epoch': 1749769200,
  'time': '2025-06-13 01:00',
  'temp_c': 20.1,
  'temp_f': 68.2,
  'is_day': 0,
  'condition': {'text': 'Clear ',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/113.

In [6]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0] # Fecha

'2025-06-13'

In [7]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]) # Hora

1

In [8]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']  #Condition

'Clear '

In [9]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c'] #Temepratura

19.9

In [10]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain'] # si llueve o no

0

In [11]:
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain'] # probabilidad de lluvia

0

# Dataframe

In [15]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain =response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [16]:
datos = []

# tqdm permite visualizar una barra de carga en los ciclos for
for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour = 'green'): 
    datos.append(get_forecast(response,i))

100%|██████████| 24/24 [00:00<?, ?it/s]


In [17]:
datos[0]

('2025-06-13', 0, 'Clear ', 19.9, 0, 0)

In [18]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia', 'prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2025-06-13,0,Clear,19.9,0,0
1,2025-06-13,1,Clear,20.1,0,0
2,2025-06-13,2,Patchy rain nearby,20.5,1,81
3,2025-06-13,3,Clear,20.4,0,0
4,2025-06-13,4,Clear,20.4,0,0
5,2025-06-13,5,Clear,20.1,0,0
6,2025-06-13,6,Clear,19.7,0,0
7,2025-06-13,7,Sunny,22.2,0,0
8,2025-06-13,8,Sunny,24.8,0,0
9,2025-06-13,9,Sunny,27.0,0,0


In [19]:
# filtrar cuando llueve
df_rain = df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora'] < 22)]
df_rain = df_rain[['Hora', 'Condicion']]
df_rain.set_index('Hora', inplace = True)

In [20]:
df_rain

,Condicion
Hora,


# Mensaje - Template

In [88]:
template = f'{str(df_rain)}'
template

'                   Condicion\nHora                        \n7         Patchy rain nearby\n8     Moderate rain at times\n9         Patchy rain nearby\n10        Patchy rain nearby\n11        Patchy rain nearby'

In [60]:
PHONE_NUMBER

'+19129001518'

# Mensaje SMS desde twilio

In [89]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages.create(
    body=template,
    from_=PHONE_NUMBER,
    to="+34744786798",
)

print(f'Mensaje Enviado {message.sid}')

Mensaje Enviado SMa14f5441daba0938ab7a72020ab433eb
